In [1]:
import os
import json

import requests
from time import sleep
from datetime import datetime, timedelta

from io import StringIO
from dotenv import load_dotenv
from pathlib import Path

import pandas as pd

from api_functions import TelegramBot
from api_functions import YandexMessengerBot

In [2]:
## Get env variables
load_dotenv()
bot_token = os.getenv("TELEGRAM_API_TOKEN")
token = os.getenv("YANDEX_TOKEN")
yam_token = os.getenv("YAM_TOKEN")

## Create Yandex messenger bot instances
chat_id_ya = '0/0/6b2bfa7e-ed2e-4be9-9a12-b753a68a4a3e'
ya_bot = YandexMessengerBot(yam_token, chat_id_ya)


url = 'https://api.direct.yandex.ru/live/v4/json/'
ReportsURL = 'https://api.direct.yandex.com/json/v5/reports'

In [3]:
### Load accounts from json
with Path('logins.json').open('r', encoding='utf-8') as f:
    logins_actual = json.load(f)

In [4]:
class YandexDirect:
    def __init__(self, token):
        """
         Initializes a new instance of the yandex direct exporter 
         with the provided token.

        Parameters: token (str) - The token for Yandex Direct API.
        """
        self.token = token
        self.url_accounts = 'https://api.direct.yandex.ru/live/v4/json/'
        self.url_reports = 'https://api.direct.yandex.com/json/v5/reports'
        self.url_campaigns = 'https://api.direct.yandex.com/json/v5/campaigns'
    def get_campaigns_spent(self, logins, date_range="YESTERDAY"):
        """
        Returns accounts spent
        """
        main_url = self.url_reports
        token = self.token
        headers = {
            "Authorization": "Bearer " + token,
            "Accept-Language": "ru",
            'skipReportHeader': "true",
            'skipColumnHeader': "true",
            'skipReportSummary': "true",
            'returnMoneyInMicros': "false"
        }
        body = {
            "params": {
                "SelectionCriteria": {},
                "FieldNames": ["Cost","CampaignName"],
                "ReportName": "CAMPAIGN_PERFORMANCE_REPORT",
                "ReportType": "CAMPAIGN_PERFORMANCE_REPORT",
                "DateRangeType": date_range,
                "Format": "TSV",
                "IncludeVAT": "YES",
                "IncludeDiscount": "NO"
            }
        }
        resultcsv = "Login,CampaignName,Costs\n"
        for Client in logins:
            # Добавление HTTP-заголовка "Client-Login"
            headers['Client-Login'] = Client
            # Кодирование тела запроса в JSON
            requestBody = json.dumps(body, indent=4)
            # Запуск цикла для выполнения запросов
            # Если получен HTTP-код 200, то содержание отчета добавляется к результирующим данным
            # Если получен HTTP-код 201 или 202, выполняются повторные запросы
            while True:
                try:
                    req = requests.post(main_url, requestBody, headers=headers)
                    req.encoding = 'utf-8'  # Принудительная обработка ответа в кодировке UTF-8
                    if req.status_code == 400:
                        print("Параметры запроса указаны неверно или достугнут лимит отчетов в очереди")
                        print(f"RequestId: {req.headers.get('RequestId', False)}")
                        print(f"JSON-код запроса: {body}")
                        print(f"JSON-код ответа сервера: \n{req.json()}")
                        break
                    elif req.status_code == 200:
                        print(f"Отчет для аккаунта {str(Client)} создан успешно")
                        print(f"RequestId: {req.headers.get('RequestId', False)}")
                        if req.text != "":
                            tempresult = req.text.split('\t')
                            resultcsv += "{},{}\n".format(Client, tempresult[0])
                        else:
                            resultcsv += "{},0\n".format(Client)
                        break
                    elif req.status_code == 201:
                        print(f"Отчет для аккаунта {str(Client)} успешно поставлен в очередь в режиме offline")
                        retryIn = int(req.headers.get("retryIn", 60))
                        print(f"Повторная отправка запроса через {retryIn} секунд")
                        print(f"RequestId: {req.headers.get('RequestId', False)}")
                        sleep(retryIn)
                    elif req.status_code == 202:
                        print("Отчет формируется в режиме офлайн")
                        retryIn = int(req.headers.get("retryIn", 60))
                        print(f"Повторная отправка запроса через {retryIn} секунд")
                        print(f"RequestId: {req.headers.get('RequestId', False)}")
                        sleep(retryIn)
                    elif req.status_code == 500:
                        print("При формировании отчета произошла ошибка. Пожалуйста, попробуйте повторить запрос позднее.")
                        print(f"RequestId: {req.headers.get('RequestId', False)}")
                        print(f"JSON-код ответа сервера: \n{req.json()}")
                        break
                    elif req.status_code == 502:
                        print("Время формирования отчета превысило серверное ограничение.")
                        print(
                            "Пожалуйста, попробуйте изменить параметры запроса - уменьшить период и количество запрашиваемых данных.")
                        print(f"JSON-код запроса: {body}")
                        print(f"RequestId: {req.headers.get('RequestId', False)}")
                        print(f"JSON-код ответа сервера: \n{req.json()}")
                        break
                    else:
                        print("Произошла непредвиденная ошибка")
                        print(f"RequestId: {req.headers.get('RequestId', False)}")
                        print(f"JSON-код запроса: {body}")
                        print(f"JSON-код ответа сервера: \n{req.json()}")
                        break

                # Обработка ошибки, если не удалось соединиться с сервером API Директа
                except ConnectionError:
                    # В данном случае мы рекомендуем повторить запрос позднее
                    print("Произошла ошибка соединения с сервером API")
                    # Принудительный выход из цикла
                    break

                # Если возникла какая-либо другая ошибка
                except:
                    # В данном случае мы рекомендуем проанилизировать действия приложения
                    print("Произошла непредвиденная ошибка")
                    # Принудительный выход из цикла
                    break
        return resultcsv

In [5]:
### Create Yandex Direct instance
yandex = YandexDirect(token)

In [6]:
campaigns_spend = yandex.get_campaigns_spent(logins_actual)

Отчет для аккаунта lost-ark-mrt создан успешно
RequestId: 8121490799349209003
Отчет для аккаунта perfect-world-mrt создан успешно
RequestId: 4073712436810758148
Отчет для аккаунта arche-age-mrt создан успешно
RequestId: 5236608835265179259
Отчет для аккаунта allodsonline-games создан успешно
RequestId: 6860634324343068475
Отчет для аккаунта atomicheart-game создан успешно
RequestId: 4967157384850408403
Отчет для аккаунта warface2016-2017warfacerwyndxg создан успешно
RequestId: 900900329164743569
Отчет для аккаунта revival-astrum создан успешно
RequestId: 5390504632829828937
Отчет для аккаунта battle-teams2 создан успешно
RequestId: 4079438152991950348


In [8]:
df = pd.read_csv(StringIO(campaigns_spend), sep=',', escapechar='\\', index_col=False)

In [9]:
campaigns_spend.head()

AttributeError: 'str' object has no attribute 'head'

In [ ]:
### Create Yandex Direct instance
yandex = YandexDirect(token)

### Create dataframe with accounts budgets
df = pd.DataFrame(yandex.accounts_budget(logins_actual))

### Create dataframe with accounts spent
budget_dict = yandex.get_account_spent(logins_actual)

### Create dataframe frome temporary dictionary
budget_dict = pd.read_csv(StringIO(budget_dict), sep=',', escapechar='\\', index_col=False)

### Filter accounts by spent for three days
budget_dict = budget_dict[(budget_dict['Costs']!= 0)]
merged_df = pd.merge(budget_dict, df, how='left', on='Login').fillna(0)

### Create column with average cost
merged_df['avg_cost'] = round(merged_df['Costs'] / 3, 2)

### Create column with days
merged_df['days'] = pd.to_numeric(merged_df['Amount'] // merged_df['avg_cost'], downcast='integer')

## Create message
message = []

### Add info to message for each account
for row in merged_df.index:
    if merged_df['days'][row] == 0 and merged_df['Amount'][row] < 5000:
        message.append(f"На аккаунте {merged_df['Login'][row]} закончился бюджет.")
    elif merged_df['days'][row] == 0 and merged_df['Amount'][row] > 5000:
        message.append(f"На аккаунте {merged_df['Login'][row]} бюджета меньше чем на день.")
    elif merged_df['days'][row] == 1:
        message.append(f"{merged_df['Login'][row]} хватит бюджета на {merged_df['days'][row]} день.")
    elif 1 < merged_df['days'][row] < 4:
        message.append(f"{merged_df['Login'][row]} хватит бюджета на {merged_df['days'][row]} дня.")

### Merge message from list to string
message = "\n\n".join(message)

## Send message
if message:
    message = f"Бюджеты в Yandex:\n {message}"
    ya_bot.send_text(message)

## Send Alert to Telegram
chat_id = '126841573'
telegram_bot = TelegramBot(bot_token, chat_id)
message_test = f"Yandex скрипт отработал"
telegram_bot.send_message(message_test)